In [ ]:
import pandas as pd
import mlrose
import numpy as np

# 참고 사이트: https://towardsdatascience.com/solving-travelling-salesperson-problems-with-python-5de7e883d847

In [ ]:
on_79 = pd.read_csv("point_79_on.csv")
on_79.head()
# 출근시간 가장 많이 탑승하는 데이터 가지고 분석
# 지금은 이렇게 하지만, 추후 출근-퇴근 둘을 섞어서 분석할 예정
# 아닌가? 다람쥐버스면 차라리 한곳을 지나는버스가 더 나을수도
# 디익스트라 사용하는게 출퇴근 둘다 사용하고, 이런 다람쥐버스는 그냥 한 지역 도는게 나을수도 있겠다.
# 조금 더 고민해볼 예정

,Unnamed: 0,이비카드정류장ID,정류장이름,위도,경도,횟수
0,316,4197607.0,서동탄역신일해피트리(중),37.199500,127.052483,317.0
1,808,4197590.0,솔빛마을.서해.쌍용아파트,37.195817,127.069633,304.0
2,1143,4116691.0,안녕농협삼거리,37.205217,127.014700,335.0
3,1649,4117227.0,수원대학교,37.213750,126.980150,506.0
4,2030,4197467.0,한빛마을(중),37.207567,127.069200,381.0


In [ ]:
#화성,병점권에 있는 정류장들만 선별
un_hsbj = on_79[on_79['경도']<127.03].index
on_79 = on_79.drop(un_hsbj)
on_79 = on_79.reset_index(drop=True)

In [ ]:
subset = on_79[['위도', '경도']]
tuple_on79 = [tuple(x) for x in subset.to_numpy()]

# 위도 경도 정보를 튜플형태 리스트로

In [ ]:
fitness_coords = mlrose.TravellingSales(coords = tuple_on79)

In [ ]:
problem_fit = mlrose.TSPOpt(length=37, fitness_fn = fitness_coords, maximize=False)

In [ ]:
best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)

print('The best state found is: \n', best_state)
# the order of visiting, the number is the index of the dataframe
print('The fitness at the best state is:\n ', best_fitness)

The best state found is: 
 [11 23  2 31 19 18 22  5 26 35  1 10 24 17  8  6 13 21 36  3 14 15 12 32
 16 20 27 29  4 33 28 34  7 25  9  0 30]
The fitness at the best state is:
  0.9866180126496746


In [ ]:
import folium 
import folium.plugins

# folium for visualization

In [ ]:
latitude = 37.136881
longitude = 127.036171

# 화성시 위도경도

In [ ]:
m = folium.Map(location=[latitude, longitude], zoom_start=12)

In [ ]:
# best_state: index 순서
# on_79 df 사용해서 folium에 길 그리기

# reindex using the tsp result
new_index = best_state
replaced = on_79.reindex(new_index)
replaced.head()

,Unnamed: 0,이비카드정류장ID,정류장이름,위도,경도,횟수
11,7623,4170264.0,대우푸르지오,37.228950,127.046050,303.0
23,24592,4151723.0,우남아파트,37.199367,127.048517,293.0
2,2030,4197467.0,한빛마을(중),37.207567,127.069200,381.0
31,32966,4170723.0,우남퍼스트빌,37.203167,127.098600,312.0
19,19151,4121008.0,이주택지.상록.경남아파트,37.207217,127.110683,338.0


In [ ]:
replaced.head()
# replaced dataframe은 TSP 알고리즘에서 방문 순서대로

,Unnamed: 0,이비카드정류장ID,정류장이름,위도,경도,횟수
11,7623,4170264.0,대우푸르지오,37.228950,127.046050,303.0
23,24592,4151723.0,우남아파트,37.199367,127.048517,293.0
2,2030,4197467.0,한빛마을(중),37.207567,127.069200,381.0
31,32966,4170723.0,우남퍼스트빌,37.203167,127.098600,312.0
19,19151,4121008.0,이주택지.상록.경남아파트,37.207217,127.110683,338.0


In [ ]:
# make list of longitude and latitude to draw routes in folium
locations = replaced[['위도', '경도']]
locationlist = locations.values.tolist()
len(locationlist)
# locationlist[0]

37

In [ ]:
import pandas as pd
import requests
from xml.etree import ElementTree
import numpy as np
import folium
i=1

for point in range(0, len(locationlist)):
    # i: 방문순서, pop-up에 커서를 대면 순서가 나온다.
    folium.Marker(locationlist[point], i).add_to(m)
    i = i+1
m

# draw lines
# add lines
folium.PolyLine(locationlist, color="purple", weight=2.5, opacity=1).add_to(m)

In [ ]:
m.save('index.html')